<a href="https://colab.research.google.com/github/JoelAbadi/DI-Bootcamp/blob/main/daily_challenge_W7_D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3


In [5]:
import zipfile

with zipfile.ZipFile("/content/Approach to Complex SQLquery Building in Kaggle.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")


In [6]:
import os

for root, dirs, files in os.walk("/content"):
    for file in files:
        print(os.path.join(root, file))


/content/database.sqlite
/content/ipl_database.sqlite
/content/Approach to Complex SQLquery Building in Kaggle.zip
/content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
/content/.config/.last_survey_prompt.yaml
/content/.config/.last_update_check.json
/content/.config/default_configs.db
/content/.config/active_config
/content/.config/.last_opt_in_prompt.yaml
/content/.config/config_sentinel
/content/.config/gce
/content/.config/logs/2025.07.01/21.03.53.252709.log
/content/.config/logs/2025.07.01/21.04.06.597104.log
/content/.config/logs/2025.07.01/21.03.20.550425.log
/content/.config/logs/2025.07.01/21.03.41.948871.log
/content/.config/logs/2025.07.01/21.03.54.626322.log
/content/.config/logs/2025.07.01/21.04.07.490711.log
/content/.config/configurations/config_default
/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/mnist_test.csv
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_te

In [12]:
import sqlite3
import pandas as pd


conn = sqlite3.connect("/content/database.sqlite")

In [13]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,Player
1,Extra_Runs
2,Batsman_Scored
3,Batting_Style
4,Bowling_Style
5,Country
6,Season
7,City
8,Outcome
9,Win_By


In [10]:
pd.read_sql("PRAGMA table_info(Player);", conn)


,cid,name,type,notnull,dflt_value,pk


Query 1 Select All from Player_Match

In [14]:
query1 = "SELECT * FROM Player_Match;"
df1 = pd.read_sql(query1, conn)
df1.head()

,Match_Id,Player_Id,Role_Id,Team_Id
0,335987,1,1,1
1,335987,2,3,1
2,335987,3,3,1
3,335987,4,3,1
4,335987,5,3,1


Query 2 Total Runs Scored by Each Batsman

In [15]:
query2 = """
SELECT P.Player_Name, SUM(BS.Runs_Scored) AS Total_Runs
FROM Ball_by_Ball AS B
JOIN Batsman_Scored AS BS ON B.Match_Id = BS.Match_Id AND B.Over_Id = BS.Over_Id AND B.Ball_Id = BS.Ball_Id AND B.Innings_No = BS.Innings_No
JOIN Player AS P ON B.Striker = P.Player_Id
GROUP BY P.Player_Name
ORDER BY Total_Runs DESC;
"""
df2 = pd.read_sql(query2, conn)
df2.head()


,Player_Name,Total_Runs
0,SK Raina,4106
1,V Kohli,4105
2,RG Sharma,3874
3,G Gambhir,3634
4,CH Gayle,3447


Query 3 Fifties and Hundreds

In [16]:
query3 = """
WITH Runs_Per_Match AS (
    SELECT B.Striker AS Player_Id,
           SUM(BS.Runs_Scored) AS Runs_In_Match
    FROM Ball_by_Ball AS B
    JOIN Batsman_Scored AS BS ON B.Match_Id = BS.Match_Id AND B.Over_Id = BS.Over_Id AND B.Ball_Id = BS.Ball_Id AND B.Innings_No = BS.Innings_No
    GROUP BY B.Match_Id, B.Striker
)
SELECT P.Player_Name,
       SUM(CASE WHEN R.Runs_In_Match BETWEEN 50 AND 99 THEN 1 ELSE 0 END) AS Fifties,
       SUM(CASE WHEN R.Runs_In_Match >= 100 THEN 1 ELSE 0 END) AS Hundreds
FROM Runs_Per_Match AS R
JOIN Player AS P ON R.Player_Id = P.Player_Id
GROUP BY P.Player_Name
ORDER BY Fifties DESC, Hundreds DESC;
"""
df3 = pd.read_sql(query3, conn)
df3.head()


,Player_Name,Fifties,Hundreds
0,DA Warner,32,2
1,G Gambhir,31,0
2,RG Sharma,29,1
3,SK Raina,28,1
4,V Kohli,27,4


Query 4 Best Bowling Figures

In [18]:
query4 = """
WITH Bowler_Match_Stats AS (
  SELECT B.Bowler AS Bowler_Id,
         B.Match_Id,
         SUM(CASE
               WHEN WT.Player_Out IS NOT NULL AND
                    (SELECT Out_Name FROM Out_Type WHERE Out_Id = WT.Kind_Out) != 'run out'
               THEN 1 ELSE 0 END) AS Wickets,
         SUM(BS.Runs_Scored + IFNULL(ER.Extra_Runs, 0)) AS Runs_Conceded
  FROM Ball_by_Ball AS B
  LEFT JOIN Wicket_Taken AS WT ON B.Match_Id = WT.Match_Id AND B.Over_Id = WT.Over_Id AND B.Ball_Id = WT.Ball_Id AND B.Innings_No = WT.Innings_No
  LEFT JOIN Batsman_Scored AS BS ON B.Match_Id = BS.Match_Id AND B.Over_Id = BS.Over_Id AND B.Ball_Id = BS.Ball_Id AND B.Innings_No = BS.Innings_No
  LEFT JOIN Extra_Runs AS ER ON B.Match_Id = ER.Match_Id AND B.Over_Id = ER.Over_Id AND B.Ball_Id = ER.Ball_Id AND B.Innings_No = ER.Innings_No
  GROUP BY B.Bowler, B.Match_Id
),
Ranked_Bowling AS (
  SELECT Bowler_Id,
         Match_Id,
         Wickets,
         Runs_Conceded,
         ROW_NUMBER() OVER(PARTITION BY Bowler_Id ORDER BY Wickets DESC, Runs_Conceded ASC) as rn
  FROM Bowler_Match_Stats
)
SELECT P.Player_Name, RB.Wickets AS Best_Wickets, RB.Runs_Conceded AS Best_Runs
FROM Ranked_Bowling RB
JOIN Player P ON P.Player_Id = RB.Bowler_Id
WHERE RB.rn = 1
ORDER BY Best_Wickets DESC, Best_Runs ASC;
"""
df4 = pd.read_sql(query4, conn)
df4.head()

,Player_Name,Best_Wickets,Best_Runs
0,Sohail Tanvir,6,15
1,A Zampa,6,19
2,A Kumble,5,6
3,I Sharma,5,12
4,SL Malinga,5,12


Query 5 Comprehensive Career Metrics

In [20]:
query5 = """
-- Combine Total Runs, Fifties, Hundreds, and Best Bowling Stats
WITH Runs AS (
    SELECT B.Striker AS Player_Id, SUM(BS.Runs_Scored) AS Total_Runs
    FROM Ball_by_Ball AS B
    JOIN Batsman_Scored AS BS ON B.Match_Id = BS.Match_Id AND B.Over_Id = BS.Over_Id AND B.Ball_Id = BS.Ball_Id AND B.Innings_No = BS.Innings_No
    GROUP BY B.Striker
),
Runs_Per_Match AS (
    SELECT B.Striker AS Player_Id,
           B.Match_Id,
           SUM(BS.Runs_Scored) AS Runs_In_Match
    FROM Ball_by_Ball AS B
    JOIN Batsman_Scored AS BS ON B.Match_Id = BS.Match_Id AND B.Over_Id = BS.Over_Id AND B.Ball_Id = BS.Ball_Id AND B.Innings_No = BS.Innings_No
    GROUP BY B.Match_Id, B.Striker
),
FiftiesHundreds AS (
    SELECT Player_Id,
           SUM(CASE WHEN Runs_In_Match BETWEEN 50 AND 99 THEN 1 ELSE 0 END) AS Fifties,
           SUM(CASE WHEN Runs_In_Match >= 100 THEN 1 ELSE 0 END) AS Hundreds
    FROM Runs_Per_Match
    GROUP BY Player_Id
),
Bowler_Match_Stats AS (
  SELECT B.Bowler AS Bowler_Id,
         B.Match_Id,
         SUM(CASE
               WHEN WT.Player_Out IS NOT NULL AND
                    (SELECT Out_Name FROM Out_Type WHERE Out_Id = WT.Kind_Out) != 'run out'
               THEN 1 ELSE 0 END) AS Wickets,
         SUM(BS.Runs_Scored + IFNULL(ER.Extra_Runs, 0)) AS Runs_Conceded
  FROM Ball_by_Ball AS B
  LEFT JOIN Wicket_Taken AS WT ON B.Match_Id = WT.Match_Id AND B.Over_Id = WT.Over_Id AND B.Ball_Id = WT.Ball_Id AND B.Innings_No = WT.Innings_No
  LEFT JOIN Batsman_Scored AS BS ON B.Match_Id = BS.Match_Id AND B.Over_Id = BS.Over_Id AND B.Ball_Id = BS.Ball_Id AND B.Innings_No = BS.Innings_No
  LEFT JOIN Extra_Runs AS ER ON B.Match_Id = ER.Match_Id AND B.Over_Id = ER.Over_Id AND B.Ball_Id = ER.Ball_Id AND B.Innings_No = ER.Innings_No
  GROUP BY B.Bowler, B.Match_Id
),
Best_Bowling AS (
  SELECT Bowler_Id,
         Wickets AS Best_Wickets,
         Runs_Conceded AS Best_Runs
  FROM (
    SELECT Bowler_Id,
           Match_Id,
           Wickets,
           Runs_Conceded,
           ROW_NUMBER() OVER(PARTITION BY Bowler_Id ORDER BY Wickets DESC, Runs_Conceded ASC) as rn
    FROM Bowler_Match_Stats
  ) AS Ranked_Bowling
  WHERE rn = 1
)
SELECT P.Player_Name,
       IFNULL(R.Total_Runs, 0) AS Total_Runs,
       IFNULL(FH.Fifties, 0) AS Fifties,
       IFNULL(FH.Hundreds, 0) AS Hundreds,
       IFNULL(BB.Best_Wickets, 0) AS Best_Bowling_Wickets,
       IFNULL(BB.Best_Runs, 0) AS Best_Bowling_Runs
FROM Player P
LEFT JOIN Runs R ON P.Player_Id = R.Player_Id
LEFT JOIN FiftiesHundreds FH ON P.Player_Id = FH.Player_Id
LEFT JOIN Best_Bowling BB ON P.Player_Id = BB.Bowler_Id
ORDER BY Total_Runs DESC;
"""
df5 = pd.read_sql(query5, conn)
df5.head()

,Player_Name,Total_Runs,Fifties,Hundreds,Best_Bowling_Wickets,Best_Bowling_Runs
0,SK Raina,4106,28,1,2,0
1,V Kohli,4105,27,4,2,24
2,RG Sharma,3874,29,1,4,7
3,G Gambhir,3634,31,0,0,0
4,CH Gayle,3447,21,5,3,21
